# Imports

In [11]:
import os
import random
import pandas as pd
from pydub import AudioSegment
from midi2audio import FluidSynth
from mido import Message, MidiFile, MidiTrack
import pygame
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import time
import cv2

midi_file = 'random.mid'
wave_file = 'output.wav'

# Note map

In [12]:
note_map = {
    0: 45,
    1: 47,
    2: 48,
    3: 50,
    4: 52,
    5: 53,
    6: 55
}

# Helpers

### get spectogram

In [13]:
def getSpectogram(wave_file, sample_rate=12_800, n_fft=2048, hop_length=256, n_mels=512):
    samples, sr = librosa.load(wave_file, sr=sample_rate)
    mel_spectrogram = librosa.feature.melspectrogram(
        y=samples, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

    return mel_spectrogram_db, sr

### get simplified spectogram

In [14]:
def getSimpleSpectogram(wave_file: str, note_map: dict, sample_rate=12_800, n_fft=2048, hop_length=256, n_mels=512):
    spec, sr = getSpectogram(wave_file, sample_rate=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    mel_frequencies = librosa.mel_frequencies(n_mels=spec.shape[0], fmin=0, fmax=sr / 2)
    note_freqs = pd.read_csv('note_freqs.csv').values
    indexes = np.array([mel_frequencies[np.abs(mel_frequencies - val).argmin()] for val in note_freqs])
    indexes = np.array([np.where(mel_frequencies == val)[0][0] for val in indexes])
    indexes = indexes[list(note_map.values())]
    new_spec = spec[indexes]
    return new_spec, sr, mel_frequencies[indexes]

### generate random midi

In [15]:
def generate_midi(midi_file: str, note_map: dict, length=32, interval=500):
    midi = MidiFile()
    track = MidiTrack()
    midi.tracks.append(track)
    note_count = len(note_map)
    

    active = np.zeros(note_count, dtype=np.byte)
    cooldown = 0
    for _ in range(length):
        cooldown += interval
        off_notes = (np.random.rand(note_count) < 0.2).astype(np.byte) & active
        for i in np.where(off_notes)[0]:
            track.append(Message('note_on', note=note_map[int(i)], velocity=0, time=cooldown))
            active[i] = 0
            cooldown = 0

        on_notes = (np.random.rand(note_count) < 0.1).astype(np.byte) & ~active
        for i in np.where(on_notes)[0]:
            track.append(Message('note_on', note=note_map[int(i)], velocity=100, time=cooldown))
            active[i] = 1
            cooldown = 0

    cooldown = interval
    for i in np.where(active)[0]:
        track.append(Message('note_on', note=note_map[int(i)], velocity=0, time=cooldown))
        active[i] = 0
        cooldown = 0

    midi.save(midi_file)

### plot simple spec

In [16]:
def plot_spec(wave_file, note_map):
    spec, sr, freqs = getSimpleSpectogram(wave_file, note_map)
    plt.figure(figsize=(10, 5))
    plt.imshow(spec, aspect='auto', interpolation='nearest', origin='lower')
    plt.yticks(ticks=np.arange(len(freqs)), labels=np.round(freqs, 2))
    plt.colorbar(format='%+2.0f dB')
    plt.show()

# Example

In [17]:
# generate_midi(midi_file, note_map, length=16, interval=300)
# midi_df = midi_to_df(midi_file)
# midi_to_audio(midi_file, wave_file)
# midi_df

In [18]:
# pygame.init()
# pygame.mixer.init()
# pygame.mixer.music.load(midi_file)
# pygame.mixer.music.play()

In [19]:
output_dir = 'my_midi'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

files = []
for i in range(100):
    mf = f'{output_dir}/{i}.mid'
    generate_midi(mf, note_map, length=1024, interval=300)
    files.append(mf)

pd.DataFrame(files, columns=['midi_filename']).to_csv('midi_files.csv', index=False)
